In [38]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, shapiro
from scipy.stats import chi2_contingency
df = pd.read_excel("thrombolyse.xlsx")


In [40]:
print("Premières lignes du fichier :")
print(df.head())

Premières lignes du fichier :
   ndossier  Age  sexe  latéralité  Tabac  alcool  Neffa  sedentarité  HTA  \
0  626/2018   48     1           1      1       0      0            0    0   
1  123/2019   66     1           1      0       0      0            0    0   
2  161/2019   37     1           1      1       0      0            0    0   
3  288/2019   53     2           1      0       0      0            0    1   
4  281/2019   68     2           1      0       0      0            0    0   

   Diabète  ...  décompensationdediabète  hypernatrémie  tbhémostase  escarre  \
0        0  ...                        0              0            0        0   
1        0  ...                        0              0            0        0   
2        0  ...                        0              0            0        0   
3        1  ...                        1              0            0        0   
4        0  ...                        0              0            0        0   

   phlébite  E

In [42]:
print("\nRésumé statistique :")
print(df.describe())


Résumé statistique :
              Age        sexe  latéralité       Tabac      alcool       Neffa  \
count  195.000000  195.000000  195.000000  195.000000  195.000000  195.000000   
mean    68.112821    1.379487    1.035897    0.353846    0.061538    0.133333   
std     14.287822    0.486508    0.186513    0.479393    0.240934    0.340810   
min     24.000000    1.000000    1.000000    0.000000    0.000000    0.000000   
25%     59.000000    1.000000    1.000000    0.000000    0.000000    0.000000   
50%     69.000000    1.000000    1.000000    0.000000    0.000000    0.000000   
75%     80.000000    2.000000    1.000000    1.000000    0.000000    0.000000   
max     96.000000    2.000000    2.000000    1.000000    1.000000    1.000000   

       sedentarité         HTA     Diabète  Dyslipidémie  ...  \
count   195.000000  195.000000  195.000000    195.000000  ...   
mean      0.579487    0.584615    0.333333      0.184615  ...   
std       0.494912    0.494057    0.472618      0.388

In [44]:
print(df.columns)

Index(['ndossier', 'Age', 'sexe', 'latéralité', 'Tabac', 'alcool', 'Neffa',
       'sedentarité', 'HTA', 'Diabète', 'Dyslipidémie', 'ACFA',
       'insuffisancecoronaire', 'valvulopathie', 'AVC', 'AIT', 'AOMI',
       'démence', 'Délai_début_symtomatologie_arrivée_URG',
       'délai_début_avis_neuro', 'délai_début_imagerie', 'scoredeglasgow',
       'NIHSS initial', 'TAS', 'TAD', 'tdmnormale', 'signesprécocestdm',
       'hypodensité franche', 'GAD', 'Plaquettes', 'TP', 'INR',
       'NIHSSàlasortie', 'TDMcàH24', 'sylviensuperf', 'sylvientotal',
       'sylvienprof', 'cérébraleant', 'cérebralpost', 'jonctionnelant',
       'jonctionnelpost', 'tronccérebral', 'cervelet', 'lacunes', 'CE',
       'oedeme', 'effetdemasse', 'engagement', 'hydrocéphalie',
       'transformation_hémorragique', 'infectionurinaire',
       'infectionpulmonaire', 'décompensationdediabète', 'hypernatrémie',
       'tbhémostase', 'escarre', 'phlébite', 'EP', 'Mortalité', 'mannitol',
       'RANKINmodifiéà3mois', 

In [46]:
#les valeurs manquantes
df.isna().sum()

ndossier               0
Age                    0
sexe                   0
latéralité             0
Tabac                  0
                      ..
EP                     0
Mortalité              0
mannitol               0
RANKINmodifiéà3mois    0
thrombolyse            0
Length: 62, dtype: int64

In [48]:
df=df.drop('ndossier',axis=1)
df.columns

Index(['Age', 'sexe', 'latéralité', 'Tabac', 'alcool', 'Neffa', 'sedentarité',
       'HTA', 'Diabète', 'Dyslipidémie', 'ACFA', 'insuffisancecoronaire',
       'valvulopathie', 'AVC', 'AIT', 'AOMI', 'démence',
       'Délai_début_symtomatologie_arrivée_URG', 'délai_début_avis_neuro',
       'délai_début_imagerie', 'scoredeglasgow', 'NIHSS initial', 'TAS', 'TAD',
       'tdmnormale', 'signesprécocestdm', 'hypodensité franche', 'GAD',
       'Plaquettes', 'TP', 'INR', 'NIHSSàlasortie', 'TDMcàH24',
       'sylviensuperf', 'sylvientotal', 'sylvienprof', 'cérébraleant',
       'cérebralpost', 'jonctionnelant', 'jonctionnelpost', 'tronccérebral',
       'cervelet', 'lacunes', 'CE', 'oedeme', 'effetdemasse', 'engagement',
       'hydrocéphalie', 'transformation_hémorragique', 'infectionurinaire',
       'infectionpulmonaire', 'décompensationdediabète', 'hypernatrémie',
       'tbhémostase', 'escarre', 'phlébite', 'EP', 'Mortalité', 'mannitol',
       'RANKINmodifiéà3mois', 'thrombolyse'],
   

In [50]:
# Initialisation des listes
vars_numeriques = []
vars_categoriques = []

# Définition du seuil de modalités
seuil_modalites = 20

# Parcours des colonnes du dataframe
for col in df.columns:
    # Vérifier le type de la colonne
    nb_modalites = df[col].nunique()
    
    # Cas où la colonne est de type 'object' ou a peu de modalités
    if df[col].dtype == 'object' or nb_modalites <= seuil_modalites:
        vars_categoriques.append(col)
    # Vérification supplémentaire : si une colonne numérique a un petit nombre de modalités
    elif df[col].dtype in ['int64', 'float64'] and nb_modalites <= seuil_modalites:
        vars_categoriques.append(col)
    else:
        vars_numeriques.append(col)

# Affichage des variables catégoriques et numériques
print("Variables numériques :", vars_numeriques)
print("Variables catégoriques :", vars_categoriques)


Variables numériques : ['Age', 'Délai_début_symtomatologie_arrivée_URG', 'délai_début_avis_neuro', 'délai_début_imagerie', 'NIHSS initial', 'TAS', 'GAD', 'Plaquettes', 'TP', 'INR', 'NIHSSàlasortie']
Variables catégoriques : ['sexe', 'latéralité', 'Tabac', 'alcool', 'Neffa', 'sedentarité', 'HTA', 'Diabète', 'Dyslipidémie', 'ACFA', 'insuffisancecoronaire', 'valvulopathie', 'AVC', 'AIT', 'AOMI', 'démence', 'scoredeglasgow', 'TAD', 'tdmnormale', 'signesprécocestdm', 'hypodensité franche', 'TDMcàH24', 'sylviensuperf', 'sylvientotal', 'sylvienprof', 'cérébraleant', 'cérebralpost', 'jonctionnelant', 'jonctionnelpost', 'tronccérebral', 'cervelet', 'lacunes', 'CE', 'oedeme', 'effetdemasse', 'engagement', 'hydrocéphalie', 'transformation_hémorragique', 'infectionurinaire', 'infectionpulmonaire', 'décompensationdediabète', 'hypernatrémie', 'tbhémostase', 'escarre', 'phlébite', 'EP', 'Mortalité', 'mannitol', 'RANKINmodifiéà3mois', 'thrombolyse']


In [52]:
df_thrombolyse_1 = df[df['thrombolyse'] == 1]
df_thrombolyse_0 = df[df['thrombolyse'] == 0]
print("Nombre de patients avec thrombolyse :", len(df_thrombolyse_1))
print("Nombre de patients sans thrombolyse :", len(df_thrombolyse_0))

Nombre de patients avec thrombolyse : 89
Nombre de patients sans thrombolyse : 106


In [54]:
# Séparer les données en fonction de la variable thrombolyse
df_thrombolyse_1 = df[df['thrombolyse'] == 1]  # Données où thrombolyse = 1
df_thrombolyse_0 = df[df['thrombolyse'] == 0]  # Données où thrombolyse = 0

# Affichage des deux tableaux
print("Tableau avec thrombolyse = 1 (Oui) :")
print(df_thrombolyse_1)  # Affiche le tableau où thrombolyse = 1

print("\nTableau avec thrombolyse = 0 (Non) :")
print(df_thrombolyse_0)  # Affiche le tableau où thrombolyse = 0


Tableau avec thrombolyse = 1 (Oui) :
     Age  sexe  latéralité  Tabac  alcool  Neffa  sedentarité  HTA  Diabète  \
0     48     1           1      1       0      0            0    0        0   
1     66     1           1      0       0      0            0    0        0   
2     37     1           1      1       0      0            0    0        0   
3     53     2           1      0       0      0            0    1        1   
4     68     2           1      0       0      0            0    0        0   
..   ...   ...         ...    ...     ...    ...          ...  ...      ...   
179   70     1           1      1       0      0            1    1        1   
182   76     2           1      0       0      0            1    1        0   
185   85     1           1      0       0      1            1    0        1   
187   75     1           1      0       0      1            1    1        0   
193   52     2           1      0       0      0            1    1        0   

     Dyslipidé

In [56]:
def test_normalite_shapiro(df):
    # Initialiser un dictionnaire pour stocker les résultats
    resultats = {}
    
    # Itérer sur chaque colonne du DataFrame
    for col in df.select_dtypes(include=['number']).columns:
        # Tester la normalité avec Shapiro-Wilk
        stat, p_val = stats.shapiro(df[col].dropna())  # Supprimer les valeurs manquantes
        
        # Ajouter les résultats au dictionnaire
        resultats[col] = {'statistique': stat, 'p-valeur': p_val}
    
    return resultats

# Appliquer la fonction sur ton DataFrame
resultats_shapiro = test_normalite_shapiro(df)

# Affichage des résultats
for col, result in resultats_shapiro.items():
    print(f"Colonne: {col}")
    print(f"  Statistique: {result['statistique']:.4f}")
    print(f"  p-valeur: {result['p-valeur']:.4f}")
    if result['p-valeur'] < 0.05:
        print("  → La variable ne suit pas une distribution normale (p < 0.05)")
    else:
        print("  → La variable suit une distribution normale (p ≥ 0.05)")
    print()


Colonne: Age
  Statistique: 0.9746
  p-valeur: 0.0013
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: sexe
  Statistique: 0.6150
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: latéralité
  Statistique: 0.1803
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: Tabac
  Statistique: 0.6046
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: alcool
  Statistique: 0.2557
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: Neffa
  Statistique: 0.4008
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: sedentarité
  Statistique: 0.6272
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: HTA
  Statistique: 0.6259
  p-valeur: 0.0000
  → La variable ne suit pas une distribution normale (p < 0.05)

Colonne: Diabète
  Statis

C:\Users\User\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [58]:
def test_mann_whitney(df, cible='thrombolyse'):
    # Résultats stockés dans un dictionnaire
    resultats = {}

    for col in df.select_dtypes(include='number').columns:
        if col != cible:  # Ne pas tester la variable cible elle-même
            # Supprimer les NaN pour chaque groupe
            x = df_thrombolyse_0[col].dropna()
            y = df_thrombolyse_1[col].dropna()
            if len(x) > 0 and len(y) > 0:
                stat, p = mannwhitneyu(x, y, alternative='two-sided')
                resultats[col] = {'statistique': stat, 'p-valeur': p}
    
    return resultats


resultats_mw = test_mann_whitney(df)

# Affichage
for var, res in resultats_mw.items():
    print(f"Variable : {var}")
    print(f"  U-statistique : {res['statistique']:.4f}")
    print(f"  p-valeur      : {res['p-valeur']:.4f}")
    if res['p-valeur'] < 0.05:
        print("  → Différence significative entre les groupes (p < 0.05)")
    else:
        print("  → Pas de différence significative (p ≥ 0.05)")
    print()


Variable : Age
  U-statistique : 5391.0000
  p-valeur      : 0.0861
  → Pas de différence significative (p ≥ 0.05)

Variable : sexe
  U-statistique : 5085.0000
  p-valeur      : 0.2653
  → Pas de différence significative (p ≥ 0.05)

Variable : latéralité
  U-statistique : 4541.0000
  p-valeur      : 0.1653
  → Pas de différence significative (p ≥ 0.05)

Variable : Tabac
  U-statistique : 4277.5000
  p-valeur      : 0.1769
  → Pas de différence significative (p ≥ 0.05)

Variable : alcool
  U-statistique : 4568.5000
  p-valeur      : 0.3650
  → Pas de différence significative (p ≥ 0.05)

Variable : Neffa
  U-statistique : 5191.5000
  p-valeur      : 0.0403
  → Différence significative entre les groupes (p < 0.05)

Variable : sedentarité
  U-statistique : 6528.0000
  p-valeur      : 0.0000
  → Différence significative entre les groupes (p < 0.05)

Variable : HTA
  U-statistique : 5110.0000
  p-valeur      : 0.2414
  → Pas de différence significative (p ≥ 0.05)

Variable : Diabète
  U-stat

In [60]:
def variables_pertinentes_non_parametriques(df, cible='thrombolyse', alpha=0.05):
    # Séparation des groupes
    g0 = df[df[cible] == 0]
    g1 = df[df[cible] == 1]

    resultats = []

    for col in df.select_dtypes(include='number').columns:
        if col != cible:
            x = g0[col].dropna()
            y = g1[col].dropna()
            
            # Vérification de normalité
            if len(x) > 3 and len(y) > 3:  # Shapiro nécessite au moins 3 points
                p_norm_x = shapiro(x)[1]
                p_norm_y = shapiro(y)[1]
                
                # Si non normal
                if p_norm_x < 0.05 or p_norm_y < 0.05:
                    stat, p_mw = mannwhitneyu(x, y, alternative='two-sided')
                    resultats.append((col, p_mw))

    # Trier par p-valeur croissante (plus petit = plus pertinent)
    resultats.sort(key=lambda x: x[1])
    return resultats

variables_pertinentes = variables_pertinentes_non_parametriques(df)

# Affichage :
print("Variables numériques non normales les plus pertinentes (triées par p-valeur) :")
for var, p in variables_pertinentes:
    print(f"- {var} : p-valeur = {p}")


Variables numériques non normales les plus pertinentes (triées par p-valeur) :
- délai_début_avis_neuro : p-valeur = 7.608088969943352e-20
- Délai_début_symtomatologie_arrivée_URG : p-valeur = 4.2531236908845225e-19
- délai_début_imagerie : p-valeur = 1.1356664139063073e-16
- hypodensité franche : p-valeur = 8.580045800150526e-12
- TDMcàH24 : p-valeur = 2.1970666812936574e-09
- sedentarité : p-valeur = 6.879020147232827e-08
- signesprécocestdm : p-valeur = 5.9851265232125935e-05
- tdmnormale : p-valeur = 0.00985374781859976
- Neffa : p-valeur = 0.04028277888499571
- mannitol : p-valeur = 0.0492398205262351
- TAD : p-valeur = 0.07181444065825872
- Age : p-valeur = 0.0860911177005302
- AOMI : p-valeur = 0.09230192436093673
- infectionpulmonaire : p-valeur = 0.09774981461683568
- INR : p-valeur = 0.10107552386634118
- NIHSS initial : p-valeur = 0.10691005250245009
- AIT : p-valeur = 0.12628772534021107
- sylvienprof : p-valeur = 0.13296465977437083
- tbhémostase : p-valeur = 0.15005448979

C:\Users\User\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [62]:
def chi2_test(df, vars_categoriques, cible='thrombolyse'):
    resultats = []

    for var in vars_categoriques:
        if var != cible:
            # Table de contingence
            table = pd.crosstab(df[var], df[cible])
            if table.shape[0] > 1 and table.shape[1] > 1:
                chi2, p, dof, expected = chi2_contingency(table)
                resultats.append((var, p))

    # Trier par pertinence (p croissante)
    resultats.sort(key=lambda x: x[1])

    return resultats

resultats_chi2 = chi2_test(df, vars_categoriques)

# Affichage :
for var, p in resultats_chi2:
    interpretation = "Significatif" if p < 0.05 else "Non significatif"
    print(f"- {var} : p = {p:.4f} → {interpretation}")


- hypodensité franche : p = 0.0000 → Significatif
- TDMcàH24 : p = 0.0000 → Significatif
- sedentarité : p = 0.0000 → Significatif
- signesprécocestdm : p = 0.0001 → Significatif
- tdmnormale : p = 0.0145 → Significatif
- Neffa : p = 0.0648 → Non significatif
- mannitol : p = 0.0786 → Non significatif
- infectionpulmonaire : p = 0.1525 → Non significatif
- AOMI : p = 0.1608 → Non significatif
- AIT : p = 0.2056 → Non significatif
- sylvienprof : p = 0.2262 → Non significatif
- Tabac : p = 0.2282 → Non significatif
- sylvientotal : p = 0.2585 → Non significatif
- oedeme : p = 0.2610 → Non significatif
- TAD : p = 0.2627 → Non significatif
- tbhémostase : p = 0.3025 → Non significatif
- HTA : p = 0.3030 → Non significatif
- latéralité : p = 0.3132 → Non significatif
- AVC : p = 0.3260 → Non significatif
- sexe : p = 0.3320 → Non significatif
- infectionurinaire : p = 0.3336 → Non significatif
- Diabète : p = 0.3342 → Non significatif
- transformation_hémorragique : p = 0.3722 → Non signi